In [5]:
import dataLoader as D
import util as U
import sys
import torch

sys.path.append('/Users/abdielfer/RNCanWork/DL4Flood/')
# subfolder import:
from model_set.models import UNetFlood
trn_listPath = '/Users/abdielfer/RNCanWork/GDLData/TestTiling/template_project/template_project_feat1_min-annot1_trn.csv'
val_listPath = '/Users/abdielfer/RNCanWork/GDLData/TestTiling/template_project/template_project_feat1_min-annot1_val.csv'

args = {'savepath':trn_listPath, 'batch_size': 4, 'num_workers': 4,'drop_last': True}

train_set = D.customDataloader(D.customDataSet(trn_listPath, True),args).getDataloader()
val_set = D.customDataloader(D.customDataSet(val_listPath),args).getDataloader()

rasterPath = '/Users/abdielfer/RNCanWork/GDLData/TestTiling/template_project/trn/VilleGatineau-1/images/ON_VilleGatineau_440125_777821867_5038038_55970885.tif'

In [2]:
import models_trainer
import losses as L
from losses import iou_binary
from torch.optim import Adam 

In [6]:
model = UNetFlood(1,1)
loss_fn = L.lovasz_hinge
optimizer = Adam(model.parameters(), lr = 0.0001)
metric = iou_binary

trainer = models_trainer.models_trainer(model,loss_fn,optimizer,metric)
trainer.set_loaders(train_set,val_set)

In [7]:
trainer.train(50)

Epoch 0 ........ ->
train Loss = 1.1150762662291527
val Loss = 1.1956783533096313
IoU per minibatch = 0.0
Epoch 1 ........ ->
train Loss = 1.0112648084759712
val Loss = 1.1360106468200684
IoU per minibatch = 0.0
Epoch 2 ........ ->
train Loss = 1.034279428422451
val Loss = 1.0971355438232422
IoU per minibatch = 0.0
Epoch 3 ........ ->
train Loss = 0.9758837074041367
val Loss = 1.0961562395095825
IoU per minibatch = 0.0
Epoch 4 ........ ->
train Loss = 1.01572647690773
val Loss = 1.027241587638855
IoU per minibatch = 0.0
Epoch 5 ........ ->
train Loss = 0.9920685663819313
val Loss = 1.0667977333068848
IoU per minibatch = 0.0
Epoch 6 ........ ->
train Loss = 0.9919039979577065
val Loss = 0.9949703216552734
IoU per minibatch = 0.0
Epoch 7 ........ ->
train Loss = 0.9919615164399147
val Loss = 1.0171867609024048
IoU per minibatch = 0.0
Epoch 8 ........ ->
train Loss = 0.9903413206338882


KeyboardInterrupt: 

In [ ]:
item = next(iter(val_set))
imag = item[0]
mask = item[1]
print('image to model input shape',imag.shape)

model.eval()
Y_hat = model(imag)
model.train()
U.plotImageAndMask(Y_hat[0][0], mask[0][0])

In [ ]:
import torch as T
from torch.autograd import Variable

In [ ]:
flated = Y_hat[0][0].view(-1)
signs = 2. * mask[0][0].float() - 1.
print(Y_hat[0][0][0:10,300:310])
print(signs[0:10,300:310])
print(signs.shape)
errors = (1. - Y_hat[0][0] * Variable(signs))
print(errors[1:10,1:10])
# U.plotImageAndMask(signs, errors)

In [ ]:
####   Apply sigmoid to the output image to compare with Mask...IMportant!!!!
y_hatSigmoided = torch.sigmoid(Y_hat[0][0])
U.plotImageAndMask(y_hatSigmoided, mask[0][0])

In [ ]:
### Visualize all batch image-mask pairs. 
# NOTE :   img, mask = next(iter(train_set))
i = 0
j = 0
for batch in train_set:
    for item in batch:
        print(f"Bathc {i}, item {j}")
        print('item',item.shape)
        j +=1
        
    # U.plotImageAndMask(batch[0][0],batch[1][0])
    i+=1
    j=0  
    print("_______") 


In [ ]:
## Ref: 

from typing import Iterable, List
import torch
import matplotlib.pyplot as plt

def plot_imgs(images: Iterable, axs: Iterable, chnls: List[int] = [2, 1, 0], bright: float = 3.):
    for img, ax in zip(images, axs):
        arr = torch.clamp(bright * img, min=0, max=1).numpy()
        rgb = arr.transpose(1, 2, 0)[:, :, chnls]
        ax.imshow(rgb)
        ax.axis('off')


def plot_msks(masks: Iterable, axs: Iterable):
    for mask, ax in zip(masks, axs):
        ax.imshow(mask.squeeze().numpy(), cmap='gray')
        ax.axis('off')

def plot_batch(batch: dict, bright: float = 3., cols: int = 4, width: int = 5, chnls: List[int] = [2, 1, 0]):

    # Get the samples and the number of items in the batch
    samples = unbind_samples(batch.copy())
    
    # if batch contains images and masks, the number of images will be doubled
    n = 2 * len(samples) if ('image' in batch) and ('mask' in batch) else len(samples)

    # calculate the number of rows in the grid
    rows = n//cols + (1 if n%cols != 0 else 0)

    # create a grid
    _, axs = plt.subplots(rows, cols, figsize=(cols*width, rows*width))  

    if ('image' in batch) and ('mask' in batch):
        # plot the images on the even axis
        plot_imgs(images=map(lambda x: x['image'], samples), axs=axs.reshape(-1)[::2], chnls=chnls, bright=bright) #type: ignore

        # plot the masks on the odd axis
        plot_msks(masks=map(lambda x: x['mask'], samples), axs=axs.reshape(-1)[1::2]) #type: ignore

    else:

        if 'image' in batch:
            plot_imgs(images=map(lambda x: x['image'], samples), axs=axs.reshape(-1), chnls=chnls, bright=bright) #type: ignore
    
        elif 'mask' in batch:
            plot_msks(masks=map(lambda x: x['mask'], samples), axs=axs.reshape(-1)) #type: ignore

In [ ]:

from pathlib import  Path
import xarray as xr
import matplotlib.pyplot as plt

root = Path('dset-s2')
assert root.exists()

train_imgs = list((root/'tra_scene').glob('*.tif'))
train_masks = list((root/'tra_truth').glob('*.tif'))

# As the images and corresponding masks are matched by name, we will sort both lists to keep them synchronized.
train_imgs.sort(); train_masks.sort()
     

idx = 0
img = xr.open_rasterio(train_imgs[idx])
mask = xr.open_rasterio(train_masks[idx])
_, axs = plt.subplots(1, 2, figsize=(15, 6))

# plot the tile
rgb = img.data[[2, 1, 0]].transpose((1, 2, 0))/3000
axs[0].imshow(rgb.clip(min=0, max=1))

# plot the mask
axs[1].imshow(mask.data.squeeze(), cmap='Blues')